In [96]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [97]:
### Read the Master file and lead variants

In [98]:
Three <- read.table(gzfile("../data/TRANSANCESTRY_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)

In [99]:
colnames(Three)
nrow(Three)

[1] "MARKERNAME"    "CHROMOSOME"    "POSITION"      "ADIPO_EA"     
 [5] "ADIPO_NEA"     "ADIPO_EAF"     "ADIPO_N"       "ADIPO_BETA"   
 [9] "ADIPO_SE"      "ADIPO_PVALUE"  "AGEN_EA"       "AGEN_NEA"     
[13] "AGEN_EAF"      "AGEN_N"        "AGEN_BETA"     "AGEN_SE"      
[17] "AGEN_PVALUE"   "METSIM_EA"     "METSIM_NEA"    "METSIM_EAF"   
[21] "METSIM_N"      "METSIM_BETA"   "METSIM_SE"     "METSIM_PVALUE"

[1] 2197980

In [100]:
## ALLELE FLIPPING

In [101]:
# Seperate out concordant alleles

In [102]:
concordant <- Three%>%filter(ADIPO_EA==METSIM_EA & METSIM_EA==AGEN_EA)

In [103]:
# Flip metsim alleles and beta if discordant with adipogen and agen

In [104]:
 flipped_metsim <- Three%>%filter(!(ADIPO_EA==METSIM_EA & METSIM_EA==AGEN_EA))%>%filter(METSIM_EA!=ADIPO_EA& METSIM_EA!=AGEN_EA)%>%filter(ADIPO_EA==AGEN_EA & ADIPO_NEA== AGEN_NEA)%>%mutate(METSIM_EA_FLIP=METSIM_NEA,
METSIM_NEA_FLIP=METSIM_EA,METSIM_BETA=-METSIM_BETA)%>%select(-METSIM_EA,-METSIM_NEA)%>%rename(METSIM_EA=METSIM_EA_FLIP,METSIM_NEA=METSIM_NEA_FLIP)

In [105]:
# Flip adipo alleles and beta if discordant with metsim and agen

In [106]:
flipped_adipo <- Three%>%filter(!(ADIPO_EA==METSIM_EA & METSIM_EA==AGEN_EA))%>%filter(ADIPO_EA!=AGEN_EA& ADIPO_EA!=METSIM_EA)%>%filter(METSIM_EA==AGEN_EA & METSIM_NEA== AGEN_NEA)%>%
mutate(ADIPO_EA_FLIP=ADIPO_NEA,ADIPO_NEA_FLIP=ADIPO_EA,ADIPO_BETA=-ADIPO_BETA)%>%
select(-ADIPO_EA,-ADIPO_NEA)%>%rename(ADIPO_EA=ADIPO_EA_FLIP,ADIPO_NEA=ADIPO_NEA_FLIP)

In [107]:
# Flip agen alleles and beta if discordant with adipo and agen

In [108]:
flipped_agen <- Three%>%filter(!(ADIPO_EA==METSIM_EA & METSIM_EA==AGEN_EA))%>%filter(ADIPO_EA!=AGEN_EA& AGEN_EA!=METSIM_EA)%>%
filter(ADIPO_EA==METSIM_EA & METSIM_NEA== ADIPO_NEA)%>%
mutate(AGEN_EA_FLIP=AGEN_NEA,AGEN_NEA_FLIP=AGEN_EA,AGEN_BETA=-AGEN_BETA)%>%
select(-AGEN_EA,-AGEN_NEA)%>%rename(AGEN_EA=AGEN_EA_FLIP,AGEN_NEA=AGEN_NEA_FLIP)

In [109]:
##concatenate all the RSIDS

In [110]:
META_THREE <- rbind(concordant,flipped_metsim,flipped_adipo,flipped_agen)

In [111]:
##Write INFO FIELD

In [112]:
write.table(META_THREE,"../data/TRANSANCESTRY_GWAS_ALLELECORRECTED.TAB",row.names=FALSE,quote=FALSE)

In [113]:
 META_THREE <- META_THREE%>%filter(ADIPO_EA==METSIM_EA & METSIM_EA==AGEN_EA & ADIPO_NEA==METSIM_NEA & METSIM_NEA==AGEN_NEA)

In [114]:
## Raw input for METAL AND MR-MEGA

In [115]:
 Adipo_mrmega <- META_THREE%>%mutate(STRAND="+")%>%select(MARKERNAME,STRAND,CHROMOSOME,POSITION,ADIPO_EA,ADIPO_NEA,ADIPO_EAF,ADIPO_N,ADIPO_BETA,ADIPO_SE,ADIPO_PVALUE)%>%
rename(EA=ADIPO_EA,NEA=ADIPO_NEA,EAF=ADIPO_EAF,N=ADIPO_N,BETA=ADIPO_BETA,SE=ADIPO_SE,PVALUE=ADIPO_PVALUE)
write.table(Adipo_mrmega,"../data/TRANSANCESTRY_ADIPOGEN_MRMEGA_RAW.TAB",row.names=FALSE,quote=FALSE)

In [116]:
 Agen_mrmega <- META_THREE%>%mutate(STRAND="+")%>%select(MARKERNAME,STRAND,CHROMOSOME,POSITION,AGEN_EA,AGEN_NEA,AGEN_EAF,AGEN_N,AGEN_BETA,AGEN_SE,AGEN_PVALUE)%>%
rename(EA=AGEN_EA,NEA=AGEN_NEA,EAF=AGEN_EAF,N=AGEN_N,BETA=AGEN_BETA,SE=AGEN_SE,PVALUE=AGEN_PVALUE)
write.table(Agen_mrmega,"../data/TRANSANCESTRY_AGEN_MRMEGA_RAW.TAB",row.names=FALSE,quote=FALSE)

In [117]:
 metsim_mrmega <- META_THREE%>%mutate(STRAND="+")%>%select(MARKERNAME,STRAND,CHROMOSOME,POSITION,METSIM_EA,METSIM_NEA,METSIM_EAF,METSIM_N,METSIM_BETA,METSIM_SE,METSIM_PVALUE)%>%
rename(EA=METSIM_EA,NEA=METSIM_NEA,EAF=METSIM_EAF,N=METSIM_N,BETA=METSIM_BETA,SE=METSIM_SE,PVALUE=METSIM_PVALUE)
write.table(metsim_mrmega,"../data/TRANSANCESTRY_METSIM_MRMEGA_RAW.TAB",row.names=FALSE,quote=FALSE)

In [118]:
## Raw input for METASOFT

In [119]:
META_SOFT <- META_THREE%>%select(MARKERNAME,ADIPO_BETA,ADIPO_SE,AGEN_BETA,AGEN_SE,METSIM_BETA,METSIM_SE)
write.table(META_SOFT,"../data/TRANSANCESTRY_METASOFT_RAW.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE)